# 圖片資料

## 0. 載入模組

In [1]:
import os
import torch
import numpy as np
import imageio as iio

In [2]:
# 圖片路徑：
image_path = "test_cat_image.jpg"

# 圖片資料夾路徑：
image_folder_path = "test_images"

## 1. 載入圖片檔

In [3]:
img_arr = iio.v3.imread(image_path)
img_arr.shape

(3024, 4032, 3)

In [4]:
h, w, c = img_arr.shape
print("H(高度)：", h)
print("W(寬度)：", w)
print("C(色彩通道數)：", c)

H(高度)： 3024
W(寬度)： 4032
C(色彩通道數)： 3


$\star$ PyTorch 中要求圖片資料張量的軸順序為 **(C, H, W)**

## 2. 調整張量各軸的排列順序

### 2-1. 單張圖片

In [5]:
# 將 img_arr 轉成 PyTorch 張量：
img_arr = np.array(img_arr, copy=True)
img = torch.from_numpy(img_arr)

# 將第 2 軸移至第 0 軸，原本的第 0 軸及第 1 軸往後移：
new_img = img.permute(2, 0, 1)
new_img.shape

torch.Size([3, 3024, 4032])

### 2-2. 多張圖片

In [6]:
# 查看圖片資料夾內圖片張數：
count_tmp = 0
for path in os.listdir(image_folder_path):
    if os.path.isfile(os.path.join(image_folder_path, path)):
        count_tmp += 1
print("圖片張數：", count_tmp)

圖片張數： 4


In [7]:
# 設定一次可以載入 N 張圖片：
batch_size = count_tmp

# 設定張量可儲存 N 張高度 3024 與寬度 4032 像素的 RGB 圖片，並且設定每個顏色通道都能用 8 位元的正整數表示：
batch = torch.zeros(batch_size, 3, h, w, dtype=torch.uint8)

In [8]:
imgs_name = [name for name in os.listdir(image_folder_path) if os.path.splitext(name)[-1] == '.jpg']

for index, img_name in enumerate(imgs_name):
    img_arr = iio.v3.imread(os.path.join(image_folder_path, img_name))
    img_arr = np.array(img_arr, copy=True)
    img_t = torch.from_numpy(img_arr)
    img_t = img_t.permute(2, 0, 1)
    # 有些圖片會有表示透明度的第 3 軸，是我們不需要的，故只保留前面 3 軸 (H,W,C):
    img_t = img_t[:3]
    batch[index] = img_t
    
batch.shape

torch.Size([4, 3, 3024, 4032])

## 3. 正規化（normalization）

$\star$ 像素值的範圍：**0～255**

### 3-1. 直接除以 255

In [9]:
batch_1 = batch.float()
batch_1 /= 225.0

### 3-2. 標準化

計算輸入資料的**平均值**和**標準差**，並調整其大小，使每個色彩通道的平均值為 0 ，標準差為 1 。

In [10]:
# 取得色彩通道數量：
n_channels = batch.shape[1]

batch_2 = batch.float()
for c in range(n_channels):
    mean = torch.mean(batch_2[:, c])               # 計算平均值
    std = torch.std(batch_2[:, c])                 # 計算標準差
    batch_2[:, c] = (batch_2[:, c] - mean) / std   # 正規化

In [11]:
batch_2

tensor([[[[-0.3430, -0.3430, -0.3261,  ...,  0.4017,  0.4186,  0.4017],
          [-0.3430, -0.3430, -0.3430,  ...,  0.4355,  0.4524,  0.4355],
          [-0.3261, -0.3430, -0.3430,  ...,  0.5032,  0.5201,  0.4863],
          ...,
          [ 0.7232,  0.7232,  0.7401,  ...,  0.1309,  0.1478,  0.1478],
          [ 0.7232,  0.7232,  0.7232,  ...,  0.1309,  0.1478,  0.1478],
          [ 0.7232,  0.7232,  0.6894,  ...,  0.1309,  0.1478,  0.1478]],

         [[-0.3067, -0.3067, -0.2888,  ...,  0.3733,  0.4090,  0.3912],
          [-0.3067, -0.3067, -0.3067,  ...,  0.4090,  0.4448,  0.4269],
          [-0.2888, -0.3067, -0.3067,  ...,  0.4806,  0.5164,  0.4806],
          ...,
          [ 1.0533,  1.0533,  1.0712,  ...,  0.1406,  0.1585,  0.1585],
          [ 1.0533,  1.0533,  1.0533,  ...,  0.1406,  0.1585,  0.1585],
          [ 1.0533,  1.0533,  1.0175,  ...,  0.1406,  0.1585,  0.1585]],

         [[-0.1178, -0.1178, -0.0997,  ...,  0.4642,  0.4460,  0.4278],
          [-0.1178, -0.1178, -